In [1]:
import cv2
import numpy as np
from PIL import Image

In [2]:
# Load image
image = cv2.imread('Test Images\image-6.png')

In [3]:
# Preprocess the image
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (5, 5), 0)

# Thresholding
_, thresh = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

In [4]:
# Find contours
contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Sort contours based on x-coordinate (left to right)
contours = sorted(contours, key=lambda ctr: cv2.boundingRect(ctr)[0])

# Filter contours and extract character bounding boxes
min_area = 100  # Minimum contour area to be considered
min_aspect_ratio = 0.2  # Minimum aspect ratio of the bounding box
characters = []
for contour in contours:
    x, y, w, h = cv2.boundingRect(contour)
    area = cv2.contourArea(contour)
    aspect_ratio = w / float(h)
    if area > min_area and aspect_ratio > min_aspect_ratio:
        characters.append((x, y, w, h))

# Extract individual characters
for i, (x, y, w, h) in enumerate(characters):
    char_image = image[y:y+h, x:x+w]

    #Adding border
    border_color = (255, 255, 255)  # White color
    boundary_size = 10
    border_image = cv2.copyMakeBorder(char_image, boundary_size, boundary_size, boundary_size, boundary_size, cv2.BORDER_CONSTANT, value=border_color)
    
    # Making image a square
    height, width, channels = border_image.shape
    size = max(height, width)
    square_image = np.ones((size, size, channels), dtype=np.uint8) * 255
    x_offset = (size - width) // 2
    y_offset = (size - height) // 2
    square_image[y_offset:y_offset+height, x_offset:x_offset+width, :] = border_image
    cv2.imwrite(f'Characters/character_{i}.jpg', square_image)
    cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)

In [5]:
# Display the output
cv2.imshow('Segmented Characters', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [6]:
from tensorflow.keras.models import load_model
digits_model = load_model('digits_model.h5')
symbols_model = load_model('arithmetic_model.h5')

In [7]:
expression=''

for j in range(0,i+1) :
    # Open the grayscale image file
    image = Image.open(f'Characters\character_{j}.jpg')  

    # Convert the image to grayscale mode
    image = image.convert("L")

    width, height = image.size

    # Resize the image to the desired input size
    resized_image = image.resize((28,28))

    # Convert the image to a NumPy array
    image_array = np.array(resized_image)

    # Normalize the pixel values (if required)
    normalized_image = image_array / 255.0

    # Reshape the image array if necessary
    reshaped_image = normalized_image.reshape((1, 28, 28, 1))

    digit_prediction = digits_model.predict(reshaped_image)
    symbol_prediction = symbols_model.predict(reshaped_image)
    
    if(np.max(symbol_prediction)>0.999 and np.max(digit_prediction)<0.9) :
        #Arithmetic symbol
        predicted_label = np.argmax(symbol_prediction, axis=1)
        predicted_symbol = None
        if(predicted_label[0]==0) : predicted_symbol = '/'
        elif (predicted_label[0]==1) : predicted_symbol = '-'
        elif (predicted_label[0]==2) : predicted_symbol = '+'
        elif (predicted_label[0]==3) : predicted_symbol = '*'
        expression+=predicted_symbol
    else :
        #Digit
        predicted_label = np.argmax(digit_prediction, axis=1)
        expression+=str(predicted_label[0])

print(expression)

1/1 [==============================] - 0s 35ms/step
17*3


In [8]:
result = eval(expression)
print("Result:", result)

Result: 51
